In [2]:
import numpy as np
import pandas as pd
import itertools
import re

In [37]:
kr_dx=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx.csv',encoding='utf-8')#,index_col=0
kr_dx.head()

,title,url,date,keyword,creator,view,likes,depart,depart1,language,text,new_text
0,1강. 디지털 트랜스포메이션(DX)에 대한 이해,https://www.youtube.com/watch?v=TIW4r4FeOZE,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,2508,29.0,DX이해,DX기초,한국어,제 1강 ‘디지털 트랜스포메이션(DX)에 대한 이해‘ 강의입니다. \n1강에서는 디...,1강 디지털 트랜스포메이션DX에 대한 이해 제 1강 디지털 트랜스포메이션DX에 대한...
1,2강. 디지털 트랜스포메이션을 위한 사전준비,https://www.youtube.com/watch?v=zttI91TWqPs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,764,11.0,DX이해,DX기초,한국어,제 2강 ‘디지털 트랜스포메이션을 위한 사전준비‘ 강의입니다. \n2강에서는 어떻게...,2강 디지털 트랜스포메이션을 위한 사전준비 제 2강 디지털 트랜스포메이션을 위한 사...
2,3강. 디지털 역량 진단,https://www.youtube.com/watch?v=dYr37tUnlvM,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,799,9.0,DX이해,DX기초,한국어,제 3강 ‘디지털 역량 진단‘ 강의입니다. \n3강에서는 디지털 트랜스포메이션을 위...,3강 디지털 역량 진단 제 3강 디지털 역량 진단 강의입니다 \n3강에서는 디지털 ...
3,4강. 목표수립 및 방향설정,https://www.youtube.com/watch?v=5IekH0xnth8,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,504,9.0,DX이해,DX기초,한국어,제 4강 ‘목표수립 및 방향설정‘ 강의입니다. \n4강에서는 디지털 트랜스포메이션을...,4강 목표수립 및 방향설정 제 4강 목표수립 및 방향설정 강의입니다 \n4강에서는 ...
4,5강. 디지털 비즈니스 모델 혁신,https://www.youtube.com/watch?v=7k4QEJNNoNs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,405,5.0,DX이해,DX기초,한국어,제 5강 ‘디지털 비즈니스 모델 혁신‘ 강의입니다. \n5강에서는 디지털 비즈니스 ...,5강 디지털 비즈니스 모델 혁신 제 5강 디지털 비즈니스 모델 혁신 강의입니다 \n...


In [38]:
kr_dx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     756 non-null    object 
 1   url       757 non-null    object 
 2   date      757 non-null    object 
 3   keyword   505 non-null    object 
 4   creator   757 non-null    object 
 5   view      757 non-null    int64  
 6   likes     742 non-null    float64
 7   depart    757 non-null    object 
 8   depart1   757 non-null    object 
 9   language  757 non-null    object 
 10  text      740 non-null    object 
 11  new_text  756 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 71.1+ KB


## 

In [9]:
import openai
import sys
import dotenv
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
import requests
import copy
from tqdm import tqdm
import time
from ast import literal_eval
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from transformers import pipeline
from transformers import GPT2TokenizerFast
from PIL import Image

import pickle
import tiktoken

api_path="C:/Users/2hy03/Python_personal/Dx추천/gpt_api.env"
load_dotenv(api_path) # .env 파일에서 환경 변수 로드
api_key = os.getenv("API_KEY") # API 키 가져오기
openai.api_key=os.getenv("API_KEY") 

In [11]:
def print_keyword(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [12]:
query = input("콘텐츠 설명: ")

콘텐츠 설명: 


In [39]:
messages = [
    {"role": "system", "content": "You are a bot that extracts keywords. We need to get rid of advertisements and bring keywords related to digital innovation. 모든 대답은 한글로."},
    {"role": "user", "content" :"{} \n 위의 내용 중, 디지털 혁신 관련 키워드만 가져와.".format(query)}
]

In [40]:
#Ratelimiterror로 나눠서 진행.
kr_dx1=kr_dx[:150]
kr_dx2=kr_dx[150:300]
kr_dx3=kr_dx[300:450]
kr_dx4=kr_dx[450:600]
kr_dx5=kr_dx[600:]

In [43]:
# 각 텍스트에 대해 키워드를 추출하여 컬럼에 할당
for idx, text in tqdm(kr_dx1['new_text'].items()):
    # 메시지 업데이트
    messages[-1]['content'] = "콘텐츠 설명: {} \n키워드: \n디지털 혁신 관련 키워드만 가져와.".format(text)
    # 키워드 추출
    keyword = print_keyword(messages)
    # key_words 컬럼에 할당
    kr_dx1.loc[idx, 'key_words'] = keyword
    kr_dx1.to_csv('kr_dx1.csv', encoding='utf-8-sig')

0it [00:00, ?it/s]<ipython-input-43-74072b51c220>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kr_dx1.loc[idx, 'key_words'] = keyword
150it [09:06,  3.64s/it]


In [44]:
# 각 텍스트에 대해 키워드를 추출하여 컬럼에 할당
for idx, text in tqdm(kr_dx2['new_text'].items()):
    # 메시지 업데이트
    messages[-1]['content'] = "콘텐츠 설명: {} \n키워드: \n디지털 혁신 관련 키워드만 가져와.".format(text)
    # 키워드 추출
    keyword = print_keyword(messages)
    # key_words 컬럼에 할당
    kr_dx2.loc[idx, 'key_words'] = keyword
    kr_dx2.to_csv('kr_dx2.csv', encoding='utf-8-sig')

0it [00:00, ?it/s]<ipython-input-44-e299a841ec4f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kr_dx2.loc[idx, 'key_words'] = keyword
150it [07:57,  3.18s/it]


In [ ]:
# 각 텍스트에 대해 키워드를 추출하여 컬럼에 할당
for idx, text in tqdm(kr_dx3['new_text'].items()):
    # 메시지 업데이트
    messages[-1]['content'] = "콘텐츠 설명: {} \n키워드: \n디지털 혁신 관련 키워드만 가져와.".format(text)
    # 키워드 추출
    keyword = print_keyword(messages)
    # key_words 컬럼에 할당
    kr_dx3.loc[idx, 'key_words'] = keyword
    kr_dx3.to_csv('kr_dx3.csv', encoding='utf-8-sig')

In [ ]:
# 각 텍스트에 대해 키워드를 추출하여 컬럼에 할당
for idx, text in tqdm(kr_dx4['new_text'].items()):
    # 메시지 업데이트
    messages[-1]['content'] = "콘텐츠 설명: {} \n키워드: \n디지털 혁신 관련 키워드만 가져와.".format(text)
    # 키워드 추출
    keyword = print_keyword(messages)
    # key_words 컬럼에 할당
    kr_dx4.loc[idx, 'key_words'] = keyword
    kr_dx4.to_csv('kr_dx4.csv', encoding='utf-8-sig')

In [ ]:
# 각 텍스트에 대해 키워드를 추출하여 컬럼에 할당
for idx, text in tqdm(kr_dx5['new_text'].items()):
    # 메시지 업데이트
    messages[-1]['content'] = "콘텐츠 설명: {} \n키워드: \n디지털 혁신 관련 키워드만 가져와.".format(text)
    # 키워드 추출
    keyword = print_keyword(messages)
    # key_words 컬럼에 할당
    kr_dx5.loc[idx, 'key_words'] = keyword
    kr_dx5.to_csv('kr_dx5.csv', encoding='utf-8-sig')

In [4]:
df1=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx1.csv',index_col=0)
df2=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx2.csv',index_col=0)
df3=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx3.csv',index_col=0)
df4=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx4.csv',index_col=0)
df5=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/kr_dx5.csv',index_col=0)

In [5]:
df=pd.concat([df1,df2,df3,df4,df5])
df

,title,url,date,keyword,creator,view,likes,depart,depart1,language,text,new_text,key_words
0,1강. 디지털 트랜스포메이션(DX)에 대한 이해,https://www.youtube.com/watch?v=TIW4r4FeOZE,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,2508,29.0,DX이해,DX기초,한국어,제 1강 ‘디지털 트랜스포메이션(DX)에 대한 이해‘ 강의입니다. \n1강에서는 디...,1강 디지털 트랜스포메이션DX에 대한 이해 제 1강 디지털 트랜스포메이션DX에 대한...,"디지털 트랜스포메이션, 생산성, 디지털 기술, 디지털 혁신, 산업DX, 디지털전환"
1,2강. 디지털 트랜스포메이션을 위한 사전준비,https://www.youtube.com/watch?v=zttI91TWqPs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,764,11.0,DX이해,DX기초,한국어,제 2강 ‘디지털 트랜스포메이션을 위한 사전준비‘ 강의입니다. \n2강에서는 어떻게...,2강 디지털 트랜스포메이션을 위한 사전준비 제 2강 디지털 트랜스포메이션을 위한 사...,"디지털 트랜스포메이션, 디지털 혁신, 산업 DX, DX 전담조직PMO, KPC 디지..."
2,3강. 디지털 역량 진단,https://www.youtube.com/watch?v=dYr37tUnlvM,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,799,9.0,DX이해,DX기초,한국어,제 3강 ‘디지털 역량 진단‘ 강의입니다. \n3강에서는 디지털 트랜스포메이션을 위...,3강 디지털 역량 진단 제 3강 디지털 역량 진단 강의입니다 \n3강에서는 디지털 ...,"디지털 역량 진단, 디지털 트랜스포메이션, 디지털 비전, 리더십, 디지털 전략, 디..."
3,4강. 목표수립 및 방향설정,https://www.youtube.com/watch?v=5IekH0xnth8,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,504,9.0,DX이해,DX기초,한국어,제 4강 ‘목표수립 및 방향설정‘ 강의입니다. \n4강에서는 디지털 트랜스포메이션을...,4강 목표수립 및 방향설정 제 4강 목표수립 및 방향설정 강의입니다 \n4강에서는 ...,"디지털 트랜스포메이션, 디지털 비전, 디지털 리더십, 디지털 거버넌스, DX 교육,..."
4,5강. 디지털 비즈니스 모델 혁신,https://www.youtube.com/watch?v=7k4QEJNNoNs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,405,5.0,DX이해,DX기초,한국어,제 5강 ‘디지털 비즈니스 모델 혁신‘ 강의입니다. \n5강에서는 디지털 비즈니스 ...,5강 디지털 비즈니스 모델 혁신 제 5강 디지털 비즈니스 모델 혁신 강의입니다 \n...,"디지털 비즈니스 모델, 디지털 비즈니스 모델 혁신, 디지털 트랜스포메이션, 비즈니스..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,"잔디(JANDI), 잔디밭처럼 편안한 직장인 위한 소통 공간!”…토스랩 김대현 대표...",https://www.youtube.com/watch?v=WluXIGfs35g&pp...,2021.1.8,NaN,V-ON Venture must go ON,103,1.0,커뮤니케이션,그룹웨어,한국어,브이온(V-ON) 벤처·스타트업 대표 인터뷰\n\n“창업은 좀 더 나은 세상을 만드...,잔디JANDI 잔디밭처럼 편안한 직장인 위한 소통 공간토스랩 김대현 대표 인터뷰 브...,"잔디, 업무협업 툴, 실시간, 창업, 벤처스타트업, 미디어, 소통, 디지털 혁신"
753,14. 조직도 만들기(멤버 관리) ㅣ 잔디 튜토리얼,https://www.youtube.com/watch?v=unFIeTrHces&pp...,2020.12.22,NaN,JANDI,2347,1.0,커뮤니케이션,그룹웨어,한국어,멤버의 탈퇴/차단 등 멤버 관리가 궁금하신가요?\n00:00 PC에서 멤버 관리하기...,14 조직도 만들기멤버 관리 ㅣ 잔디 튜토리얼 멤버의 탈퇴차단 등 멤버 관리가 궁금...,"조직도, 멤버 관리, 무료 플랜, 유료 플랜, 모바일, 디지털 혁신"
754,협업툴 CTO의 협업 A to Z ⎮ 잔디가 자라는 방법 ep.2,https://www.youtube.com/watch?v=Ercl18BvRaU&pp...,2022.4.22,#JANDI,JANDI,608,11.0,커뮤니케이션,그룹웨어,한국어,#잔디 #JANDI #협업툴\n🌱 잔디가 자라는 방법\n✅ 언제 어디서나 가장 쉬운...,협업툴 CTO의 협업 A to Z 잔디가 자라는 방법 ep2 잔디 JANDI 협업...,"협업툴, 메신저, 업무용 커뮤니케이션 툴, 핵심 기능, 업무 몰입, 생산성, 협업,..."
755,"카카오톡? 잔디, 플로우, 네이버웍스, 노션... 우리 기업에 맞는 협업툴과 업무툴은?",https://www.youtube.com/watch?v=KTy1VZ9oMcw&pp...,2022.10.17,NaN,가인지TV,4066,66.0,커뮤니케이션,그룹웨어,한국어,✅ 경영벙커 강의파일과 자료 다운받기 👉 https://bit.ly/3CVqZX5\...,카카오톡 잔디 플로우 네이버웍스 노션 우리 기업에 맞는 협업툴과 업무툴은 경영벙커...,"카카오톡, 잔디, 플로우, 네이버웍스, 노션, 협업툴, 업무툴, 컨설팅회사, 경영 ..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757 entries, 0 to 756
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      756 non-null    object 
 1   url        757 non-null    object 
 2   date       757 non-null    object 
 3   keyword    505 non-null    object 
 4   creator    757 non-null    object 
 5   view       757 non-null    int64  
 6   likes      742 non-null    float64
 7   depart     757 non-null    object 
 8   depart1    757 non-null    object 
 9   language   757 non-null    object 
 10  text       740 non-null    object 
 11  new_text   756 non-null    object 
 12  key_words  757 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 82.8+ KB


In [7]:
df.loc[79,'title']='인공지능 시대의 브랜드 전략은? 마케팅에서 주목해야 할 #AI트랜스포메이션 l 함샤우트글로벌 chatGPT 시리즈 1화 #chatgpt #AI마케팅'

In [8]:
#전처리 전 데이터 복사
pre_df=df
# 정규표현식을 사용하여 전처리
pre_df['key_words'] = pre_df['key_words'].str.replace('-', ' ')
pre_df['key_words'] = pre_df['key_words'].apply(lambda x: re.sub(r'\n', ',', x))
# 결과 확인
print(pre_df['key_words'])

0          디지털 트랜스포메이션, 생산성, 디지털 기술, 디지털 혁신, 산업DX, 디지털전환
1      디지털 트랜스포메이션, 디지털 혁신, 산업 DX, DX 전담조직PMO, KPC 디지...
2      디지털 역량 진단, 디지털 트랜스포메이션, 디지털 비전, 리더십, 디지털 전략, 디...
3      디지털 트랜스포메이션, 디지털 비전, 디지털 리더십, 디지털 거버넌스, DX 교육,...
4      디지털 비즈니스 모델, 디지털 비즈니스 모델 혁신, 디지털 트랜스포메이션, 비즈니스...
                             ...                        
752         잔디, 업무협업 툴, 실시간, 창업, 벤처스타트업, 미디어, 소통, 디지털 혁신
753                조직도, 멤버 관리, 무료 플랜, 유료 플랜, 모바일, 디지털 혁신
754    협업툴, 메신저, 업무용 커뮤니케이션 툴, 핵심 기능, 업무 몰입, 생산성, 협업,...
755    카카오톡, 잔디, 플로우, 네이버웍스, 노션, 협업툴, 업무툴, 컨설팅회사, 경영 ...
756                              잔디 튜토리얼, 멘션, PC, Mobile
Name: key_words, Length: 757, dtype: object


In [9]:
pre_df[20:30]['key_words']

20      Self Service BI,  Power BI,  MS 데이터 시각화 툴,  ...
21    2021 데이터 컨퍼런스, 전사 분석, BI 트렌드, 도입사례, 하이브리드 데이터 ...
22    디지털 혁신, 제조산업, ERP 운영, MDM 통합, Cloud 전환, 데이터 분석...
23          창업, 비즈니스 모델, 구체화, 비즈니스 모델 캔버스, 핵심요소, 디지털 혁신
24    멤버십, 기업분석, 투자 격언, 구독자, 이벤트, 후원, 가치투자101, 비즈니스 ...
25                      SNS마케팅, 비즈니스 모델, 사업화 전략, 디지털 혁신
26    삼국지 전략판, 모바일 게임, 매출 10위, SLG, PVP, 흥행, 게임, 전세계...
27                         IT, 비즈니스 모델, 스타트업, 구체화, 비즈니스
28        비즈니스 모델, 구체화, 전략, 창업단계, 지니 변호사, 10단계, 디지털 혁신.
29    외국계, BM, 브랜드 마케터, 브랜드 마케팅, 마케팅, 디지털 혁신, 존슨앤존슨,...
Name: key_words, dtype: object

In [ ]:
pre_df.to_csv('kr_dx_concat_pre.csv',encoding='utf-8-sig') 